In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import tensorflow as tf
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from conv_util import *

## Storks, 1D convolution

In [2]:
data = pd.read_csv('./storks_obs_train.csv', header=None)

X = np.array(data.drop(columns=120))
Y = np.array(data[120])

le = preprocessing.LabelEncoder()
le.fit(np.unique(Y))
Y = le.transform(Y)

train_x, test_x, train_y, test_y = train_test_split(X, Y, test_size=0.2)

In [3]:
def sig_reshape (data):
    
    x_axis = np.append(data[0::3], np.zeros(9))
    y_axis = np.append(data[1::3], np.zeros(9))
    z_axis = np.append(data[2::3], np.zeros(9))
    
    out = np.stack((x_axis, y_axis, z_axis))
    
    return out.reshape([7, 7, 3])

In [4]:
train_x_re = []
test_x_re = []

for x in train_x:
    train_x_re.append(sig_reshape(x))
    
for x in test_x:
    test_x_re.append(sig_reshape(x))

In [5]:
train_x_re = np.array(train_x_re)
test_x_re = np.array(test_x_re)

In [14]:
train_x_re = train_x_re.astype(np.float32)
test_x_re = test_x_re.astype(np.float32)

In [15]:
train_x_re.shape

(1396, 7, 7, 3)

In [16]:
def build_model_1(
    x,
    keep_rate):

    # Conv layer #1
    conv1 = conv_layer(
        x=x,
        input_channels=3, 
        filter_height=5, 
        filter_width=5, 
        num_filters=32, 
        stride_y=1, 
        stride_x=1, 
        padding='SAME', 
        name='conv1')
    
#     pool1 = max_pool(
#         x=conv1,
#         filter_height=2,
#         filter_width=2, 
#         stride_y=2, 
#         stride_x=2, 
#         padding='VALID', 
#         name='pool1')

    # Conv layer #2
    conv2 = conv_layer(
        x=conv1,
        input_channels=32, 
        filter_height=5, 
        filter_width=5, 
        num_filters=64, 
        stride_y=1, 
        stride_x=1, 
        padding='SAME', 
        name='conv2')

    # Fc layer #3
    flat = tf.reshape(
        tensor=conv2,
        shape=[-1, 7*7*64])

    fc3 = fc_layer(
        x=flat,
        num_input=7*7*64,
        num_output=1024, 
        name='fc3',
        is_relu=True)

#     drop = dropout(
#         x=fc5,
#         keep_rate=keep_rate)

    # Logit layer #6
    fc4 = fc_layer(
        x=fc3,
        num_input=1024,
        num_output=6, 
        name='fc4',
        is_relu=False)

    return fc4

In [17]:
def build_model_2(
    x,
    keep_rate):

    # Conv layer #1
    conv1 = conv_layer(
        x=x,
        input_channels=3, 
        filter_height=3, 
        filter_width=3, 
        num_filters=32, 
        stride_y=1, 
        stride_x=1, 
        padding='SAME', 
        name='conv1')
    
    # Conv layer #2
    conv2 = conv_layer(
        x=conv1,
        input_channels=32, 
        filter_height=3, 
        filter_width=3, 
        num_filters=64, 
        stride_y=1, 
        stride_x=1, 
        padding='SAME', 
        name='conv2')

    # Fc layer #3
    flat = tf.reshape(
        tensor=conv2,
        shape=[-1, 7*7*64])

    fc3 = fc_layer(
        x=flat,
        num_input=7*7*64,
        num_output=1024, 
        name='fc3',
        is_relu=True)

#     drop = dropout(
#         x=fc5,
#         keep_rate=keep_rate)

    # Logit layer #6
    fc4 = fc_layer(
        x=fc3,
        num_input=1024,
        num_output=6, 
        name='fc4',
        is_relu=False)

    return fc4

In [18]:
# Misc
LOG_DIR = './log/'

# Training Parameters
learning_rate = 0.0015
num_epochs = 10
num_steps = 700
batch_size = 64
display_step = 50

# Network Parameters
num_classes = 6
keep_rate = 1

In [19]:
model_graph = tf.Graph()
with model_graph.as_default():
    
    keep_prob = tf.placeholder(tf.float32)
    
    # Dataset Preparation
    trn_ds = tf.data.Dataset.from_tensor_slices((train_x_re, train_y)).shuffle(128).repeat().batch(batch_size)
    test_ds = tf.data.Dataset.from_tensor_slices((test_x_re, test_y)).shuffle(128).repeat().batch(batch_size)
    
    iterator = tf.data.Iterator.from_structure(trn_ds.output_types, trn_ds.output_shapes)
    x, y = iterator.get_next()
    
    training_init = iterator.make_initializer(trn_ds)
    testing_init = iterator.make_initializer(test_ds)
    
    # Logits and Predictions
    logits = build_model_2(x, keep_prob) 
    prediction = {'classes': tf.argmax(logits, axis=1), 
                  'prob': tf.nn.softmax(logits, name='prob')}
    
    # Loss and optimizer
    with tf.variable_scope('cross_entropy_loss'):
        loss_op = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=y))
        tf.summary.scalar('cross_entropy_loss', loss_op)
    
    with tf.variable_scope('train'):
        optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
        train_op = optimizer.minimize(loss_op, global_step=tf.train.get_global_step())
    
    # Evaluation
    with tf.variable_scope('accuracy'):
        correct_pred = tf.equal(prediction['classes'], y)
        accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
        tf.summary.scalar('accuracy', accuracy)
    
    # Global Initializer
    initializer = tf.global_variables_initializer()
    
    # Summary
    summary = tf.summary.merge_all()

In [20]:
with tf.Session(graph=model_graph) as sess:
    
    # Run Global Initializer
    sess.run(initializer)
    
    sess.run(training_init)
    for step in range(1, num_steps+1):
        # Run optimization op (backprop)
        sess.run(train_op, feed_dict={keep_prob: keep_rate})
        if step % display_step == 0 or step == 1:
            # Calculate batch loss and accuracy
            loss, acc = sess.run([loss_op, accuracy], feed_dict={keep_prob: 1.0})
            print("Step " + str(step) + ", Minibatch Loss= " + \
                  "{:.4f}".format(loss) + ", Training Accuracy= " + \
                  "{:.3f}".format(acc))

    print("Optimization Finished!")

    sess.run(testing_init)
    print("Testing Accuracy:", \
        sess.run(accuracy, feed_dict={keep_prob: 1.0}))

Step 1, Minibatch Loss= 6.7672, Training Accuracy= 0.438
Step 50, Minibatch Loss= 0.5484, Training Accuracy= 0.812
Step 100, Minibatch Loss= 0.4004, Training Accuracy= 0.875
Step 150, Minibatch Loss= 0.1782, Training Accuracy= 0.953
Step 200, Minibatch Loss= 0.3299, Training Accuracy= 0.906
Step 250, Minibatch Loss= 0.2266, Training Accuracy= 0.938
Step 300, Minibatch Loss= 0.2615, Training Accuracy= 0.891


KeyboardInterrupt: 